
# Tremor location at Piton de la Fournaise
This example shows how to calculate the network response function and locate 
the tremor associated with the 21 June 2014 eruption of the Piton de la Fournaise.

Seismic data from 6 stations (only vertical component, downsampled to 20 Hz) are provided for the time period 1 June - 1 July 2014, covering both the pre-eruptive period and the effusive eruption which occurred on the south eastern flank of the volcano.

Figures of the current notebook, from top to bottom:  
#. Seismic waveform from the vertical component of station FOR, closest to the effusive eruption site  
#. Spectral width, between 0.5Hz and 10Hz  
#. Average spectral width between 0.3Hz and 5Hz (tremor frequency range)  
#. Network response function  
#. Likelihood 3-D location  
#. Unshifted (before calculating the beam) and shifted (corresponding to the maximum of the beam) cross-correlation envelopes

In [ ]:
!pip install covseisnet

In [ ]:
## IMPORTS

import covseisnet as csn
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import numpy as np
import os

from datetime import datetime, timedelta
from display import dem

In [ ]:
starttime = datetime(2014, 6, 20, 21, 0)

In [ ]:
## READ DATA

############
# Parameters
############
starttime = datetime(2014, 6, 20, 20, 0)
endtime = starttime + timedelta(minutes=2*60)
stations_list = [
     ['BON', 55.70921, -21.23971, 2.552],
     ['CRA', 55.75783, -21.22336, 0.998],
     ['CSS', 55.68451, -21.24642, 2.193],
     ['FJS', 55.72229, -21.229493, 2.123],
     ['FOR', 55.718708, -21.261867, 2.049],
     ['FRE', 55.695393, -21.201837, 1.775],
     ['GBS', 55.77934, -21.27387, 0.471],
     ['GPN', 55.75247, -21.23979, 1.413],
     ['GPS', 55.76162, -21.26741, 1.004],
     ['HIM', 55.720224, -21.211121, 1.958],
     ['PRA', 55.707912, -21.291698, 2.009],
     ['SNE', 55.7179, -21.239115, 2.505]]



net = 'PF'
year = starttime.year
julday = starttime.timetuple().tm_yday

data_path = 'XXX/SDS/'
data_file = os.path.join(data_path,'{:d}'.format(year),'{:s}'.format(net),
             '{:s}','HHZ.D','{:s}.'.format(net)+'{:s}.00.HHZ.D.'+'{:d}.{:03d}'.format(year, julday))

############

# List of data files
datafiles_list = []
for station in stations_list:
    datafiles_list.append(data_file.format(str(station[0]),str(station[0])))
# Stream object
stream = csn.arraystream.ArrayStream()
# Read if file exists
for datafile in datafiles_list:
    try:
        os.path.isfile(datafile)
        stream += csn.arraystream.read(datafile)
    except:
        print('No file {:s}'.format(datafile))
print(stream)

In [ ]:
## DATA PRE-PROCESSING

############
# Parameters
############
# Frequency
fdatafilter = [0.01, 9]
# Windows / Subwindows
window_duration_sec = 60
window_average = 16
# Spectral pre-processing
preprocess_domain = 'spectral'
preprocess_method = 'onebit'
preprocess_duration_sec = window_duration_sec * (window_average+1) * 0.5
############


# Pre-processing
stream.merge(method=1, interpolation_samples=0, fill_value=0)
stream.cut(starttime=starttime, endtime=endtime, pad=True, fill_value=0)
stream.detrend(type='demean')
stream.filter(type='bandpass', freqmin=fdatafilter[0], freqmax=fdatafilter[1], zerophase=True)
stream.sort(keys=['station'])
sampling_rate = stream[0].stats.sampling_rate
stream.preprocess(domain=preprocess_domain, method=preprocess_method, window_duration_sec=preprocess_duration_sec)
stream.cut(starttime=starttime, endtime=endtime, pad=True, fill_value=0)
print(stream)

In [ ]:
## CALCULATE SPECTRAL WIDTH

# Calculate network covariance matrix
times, frequencies, covariances = csn.covariancematrix.calculate(stream, window_duration_sec, window_average)
# Calculate spectral width
spectral_width = covariances.coherence(kind="spectral_width")

In [ ]:
print(covariances.shape)

In [ ]:
## PLOT SPECTRAL WIDTH

############
# Parameters
############
# Spectral with filtering frequency limits (depends on the target signal)
sw_freq_min = 0.3
sw_freq_max = 5.0
# Station to plot
station_plot = ['FOR', 'HHZ']
############


# Averaged spectral width value
i_freq_low = np.argmin(np.abs(frequencies - sw_freq_min))
i_freq_high = np.argmin(np.abs(frequencies - sw_freq_max))
spectral_width_average = np.mean(spectral_width[:, i_freq_low:i_freq_high], axis=1)

# Plot seismic trace from one chosen station and spectral width with its average
fig, ax = plt.subplots(3, constrained_layout=True, figsize=(10, 8))

# Seismic trace
trace_plot = stream.select(station=station_plot[0], channel=station_plot[1])[0].data
duration_min = (endtime - starttime).total_seconds() / 60
ax[0].plot(np.linspace(0, duration_min, len(trace_plot)), trace_plot, "k")
ax[0].set_title("Vertical channel of station {:s}".format(station_plot[0]))
ax[0].set_ylabel("{:s}.{:s} (counts)".format(station_plot[0], station_plot[1]))
ax[0].set_xlim([0, duration_min])

# Spectral width
img = ax[1].imshow(spectral_width.T,
                   origin="lower",
                   cmap="RdYlBu",
                   interpolation="none",
                   extent=[0, duration_min, 0, sampling_rate],
                   aspect="auto")
ax[1].axhline(sw_freq_min, ls='--', lw=3, c='k')
ax[1].axhline(sw_freq_max, ls='--', lw=3, c='k')
ax[1].set_ylim([0.1, sampling_rate / 2])
ax[1].set_ylabel("Frequency (Hz)")
ax[1].set_yscale("log")
ax[1].set_title("Covariance matrix spectral width")
plt.colorbar(img, ax=ax[1]).set_label("Spectral Width")

# Averaged spectral width
ax[2].plot(np.linspace(0, duration_min, len(spectral_width_average)), spectral_width_average, "k")
ax[2].set_title("Averaged spectral width between {:.1f}Hz and {:.1f}Hz".format(sw_freq_min, sw_freq_max))
ax[2].set_ylabel("Averaged value")
ax[2].set_xlim(0, duration_min)
ax[2].set_ylim(np.min(spectral_width), np.max(spectral_width))
ax[2].set_xlabel("Minutes")


In [ ]:
## COMPUTE NETWORK RESPONSE FUNCTION (NRF)

############
# Parameters
############
# Path to traveltime grids
traveltime_filepath = "XX/PdF/ttimes/"  # path to traveltime grids

# grid extent Longitude: 55.67° to 55.81° (145 points), Latitude: -21.3° to -21.2° (110 points)
lon_min = 55.67
lon_max = 55.81
lat_min = -21.3
lat_max = -21.2
depth_min = -2.6  # edifice summit (km)
depth_max = 3.0  # max depth of traveltime grids (km)



# Correlation filtering
fxcorr_low_pass = 0.3
fxcorr_high_pass = 5.0
# Correlation Smoothing
sigma = 20
############

# Eigenvector decomposition - covariance matrix filtered by the 1st eigenvector to show the dominant source
covariance_1st = covariances.eigenvectors(covariance=True, rank=0)

# Extract cross-correlations
lags, correlation = csn.correlationmatrix.cross_correlation(covariance_1st, sampling_rate)

# Load traveltimes grid from npy pre-calculated files
traveltimes = csn.traveltime.TravelTime(stream, traveltime_filepath)

# Initiate beam object and set geographical extent of grid
nwin = correlation.nwin()  # number of time windows
beam = csn.beam.Beam(nwin, traveltimes)
beam.set_extent(lon_min, lon_max, lat_min, lat_max, depth_min, depth_max)

# Loop through all time windows and calculate likelihood and nrf for each window
for i in range(0, nwin):
    print("Processing window {:d} of {:d}".format(i, nwin-1))

    correl = correlation[i]

    # Correlation filtering
    correl = correl.bandpass(fxcorr_low_pass, fxcorr_high_pass, sampling_rate)

    # Correlation enveloppe
    correl = correl.hilbert_envelope()
    
    # Envelope smoothing
    correl = correl.smooth(sigma=sigma)

    beam.calculate_likelihood(correl, sampling_rate, i)
    beam.calculate_nrf(i)

    beam_max = beam.max_likelihood(i)
    print("Maximum likelihood at Lon={:.2f}, Lon={:.2f}, Depth={:.2f} km".format(beam_max[0],
                                                                                 beam_max[1],
                                                                                 beam_max[2]))

In [ ]:
## PLOT NRF

fig, ax = plt.subplots(1, constrained_layout=True, figsize=(10, 3))
ax.plot(np.linspace(0, duration_min, nwin), beam.nrf, "k")
t=np.linspace(0, duration_min, nwin)
ax.set_title("Network response function (NRF)")
ax.set_ylabel("NRF, unormalized")
ax.set_xlim(0, duration_min)
ax.set_xlabel("Minutes")

In [ ]:
## PLOT 3-D LIKELIHOOD

############
# Parameters
############
# Choose the window index for plotting likelihood
i_win = 5
# Dem
demfile = "XX/PdF/dem/dem.asc"
depth_plot_lim = -3  # in km
# Eruption sites

############


# Extract max likelihood position
beam_max = beam.max_likelihood(i_win)
x_max = beam_max[0]
y_max = beam_max[1]
z_max = beam_max[2]

# Select likelihood
likelihood_xyz = beam.likelihood[i_win, :, :, :]

# Normalize likelihood betwwen 0 and 1
likelihood_xyz = (likelihood_xyz - likelihood_xyz.min()) / (likelihood_xyz.max() - likelihood_xyz.min())

# Take slices at point of max likelihood
i_max, j_max, k_max = np.unravel_index(likelihood_xyz.argmax(), likelihood_xyz.shape)
likelihood_xy = likelihood_xyz[:, :, k_max]
likelihood_xz = likelihood_xyz[:, j_max]
likelihood_yz = likelihood_xyz[i_max]

# Create custom discrete colormap for likelihood
custom_cmap = plt.cm.get_cmap("RdYlBu_r")(np.linspace(0, 1, 12))
for i in range(4):
    custom_cmap[i, :] = [1, 1, 1, 1]
for i in range(1, 12):
    custom_cmap[i, -1] = np.sqrt(i / 12)
custom_cmap = mcolors.LinearSegmentedColormap.from_list("RdYlBu_r", custom_cmap, N=16)

# Create figure
gs = dict(width_ratios=[2.5, 1], height_ratios=[2, 1], wspace=0.05, hspace=0.01)
fig, ax = plt.subplots(2, 2, constrained_layout=True, figsize=(7, 6), gridspec_kw=gs, dpi=100)
ax = ax.ravel()

# MAP VIEW
# Likelihood
ax[0].imshow(likelihood_xy.T, interpolation="none", origin="lower", cmap=custom_cmap, aspect="auto",
    extent=[lon_min, lon_max, lat_min, lat_max], vmin=0.0)
# Plot stations
for station in stations_list:
    ax[0].plot(station[1], station[2], "v", mfc='k', mec='k', mew=0.4, ms=10)
    ax[0].text(station[1]+0.005, station[2]-0.001, station[0])
# Plot eruption sites
#for eruption in eruptions_list:
#    ax[0].plot(eruption[0], eruption[1], "o", mfc='lime', mec='k', mew=0.4, ms=6)
# Max likelihood position
ax[0].plot(x_max, y_max, "*", mfc='w', mec='k', mew=0.4, ms=10)
    
# DEPTH VIEWS

# Extract dem infos
dem_lon, dem_lat, dem_alt = dem.read(demfile, depth_factor=-1e-3)
dem_alt_lon = np.nanmin(dem_alt, axis=0)
dem_alt_lat = np.nanmin(dem_alt, axis=1)

# DEPTH-LONGITUDE VIEW
# Likelihood
ax[2].imshow(likelihood_xz.T, interpolation="none", origin="upper", cmap=custom_cmap, aspect="auto",
    extent=[lon_min, lon_max, depth_max, depth_min], vmin=0.0)
# Crop out data above topo
ax[2].fill_between(dem_lon, depth_plot_lim-1, dem_alt_lon, facecolor='w', edgecolor='k', lw=.4)
# Max likelihood position
ax[2].plot(x_max, z_max, "*", mfc='w', mec='k', mew=0.4, ms=10)

# DEPTH-LATITUDE VIEW
# Likelihood
img_yz = ax[1].imshow(likelihood_yz, interpolation="none", origin="lower", cmap=custom_cmap, aspect="auto",
    extent=[depth_min, depth_max, lat_min, lat_max], vmin=0.0)
# Crop out data above topo
ax[1].fill_betweenx(dem_lat, depth_plot_lim-1, dem_alt_lat, facecolor='w', edgecolor='k', lw=.4)
# Max likelihood position
ax[1].plot(z_max, y_max, "*", mfc='w', mec='k', mew=0.4, ms=10)

# Cosmetics
ax[0].set_xticklabels([])
ax[0].set_ylabel("Latitude")
ax[1].set_xlabel("Depth (km)")
ax[1].set_yticklabels([])
ax[1].set_xlim([depth_plot_lim, depth_max])
ax[1].set_ylim([lat_min, lat_max])
ax[2].set_xlabel("Longitude")
ax[2].set_ylabel("Depth (km)")
ax[2].set_xlim([lon_min, lon_max])
ax[2].set_ylim([depth_max, depth_plot_lim])

# Colorbar
cb = plt.colorbar(img_yz, cax=ax[3], orientation='horizontal')
cb.set_label('Likelihood')
cb.set_ticks([0, 1 / 2, 1])

In [ ]:
## PLOT CROSS-CORRELATIONS ENVELOPES

############
# Parameters
############
stack_lim = [0, 0.125]
############

# Create figure
gs = dict(width_ratios=[1, 1], height_ratios=[4, 1], wspace=0.10, hspace=0.10)
fig, ax = plt.subplots(2, 2, figsize=(6, 10), gridspec_kw=gs, dpi=100)

# Index
idx = 0

# Loop over unshifted and shifted cases
for xcorr_label, xcorr in (("Unshifted envelopes", beam.correlation_unshifted[i_win]),
    ("Shifted envelopes", beam.correlation_shifted[i_win])):
    
    # Extract
    lag_max = np.max(lags) / 2
    xcorr_max = np.nanmax(xcorr)

    # Plot
    n_xcorr = xcorr.shape[0]
    for d, x in zip(range(1, n_xcorr + 1), xcorr):
        x = 0.5 * np.abs(x) / xcorr_max
        ax[0, idx].fill_between(lags, -x + d, x + d, facecolor="b", alpha=0.5)
    ax[1, idx].plot(lags, np.nansum(xcorr, axis=0), c="b", lw=1.0)

    # Cosmetics
    ax[0, idx].set_title(xcorr_label, loc="left", va="bottom", size=10)
    ax[0, idx].set_xlim([-lag_max, lag_max])
    ax[0, idx].set_ylim([0, n_xcorr + 1])
    ax[0, idx].set_yticks(range(1, n_xcorr + 1))
    ax[0, idx].set_xticklabels([])
    ax[0, idx].set_yticklabels([])
    ax[1, idx].set_title("Envelopes stack", loc="left", va="bottom", size=10)
    ax[1, idx].set_xlim([-lag_max, lag_max])
    ax[1, idx].set_ylim(np.array(stack_lim)*2)
    ax[1, idx].set_xlabel("Lag (sec)", fontsize=10)
    ax[0, idx].tick_params(which="both", direction="out", labelsize=7)
    ax[1, idx].tick_params(which="both", direction="out", labelsize=7)
    
    # Index incrementation
    idx += 1

# Cosmetics
ax[0, 0].set_ylabel("Station pairs", fontsize=10)
ax[1, 0].set_ylabel("Stack", fontsize=10)
ax[1, 1].set_yticklabels([])
    